In [1]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.append(os.path.abspath('../src'))

from data.load_data import load_images_from_folder, load_video


folder_path = '../data/raw/mobil'
images = load_images_from_folder(folder_path)
print(f"Jumlah gambar mobil yang dimuat: {len(images)}")

# Memuat video mobil
video_path = '../data/raw/videosm.mp4'
frames = load_video(video_path)
print(f"Jumlah frame dalam video: {len(frames)}")

Jumlah gambar mobil yang dimuat: 153
Jumlah frame dalam video: 3606
